In [79]:
!spacy download en

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/urllib3/contrib/pyopenssl.py", line 453, in wrap_socket
    cnx.do_handshake()
  File "/anaconda3/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1915, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "/anaconda3/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1639, in _raise_ssl_error
    raise SysCallError(errno, errorcode.get(errno))
OpenSSL.SSL.SysCallError: (60, 'ETIMEDOUT')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 839, in _validate_conn
    conn.connect()
  File "/anaconda3/lib/python3.7/sit

In [78]:
import nltk
import gensim
import textblob
import spacy
nlp = spacy.load('en')
from wordcloud import wordcloud
import string
import re

from nltk.corpus import stopwords,wordnet
stopwords_ = stopwords.words('english')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer


pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [68]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [69]:
k = pd.DataFrame()
k["train"] = round(100*train.isna().sum()/len(train),2)
k["test"] = round(100*test.isna().sum()/len(test),2)
k.T

,id,label,tweet
train,0.0,0.0,0.0
test,0.0,NaN,0.0


In [70]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [71]:
def get_url(text):
    u1 = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',text)
    #text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
    
#     u2 = re.findall('[^ ]+\.[com]','',text)
#     text = re.sub('[^ ]+\.[com]','',text)
    return len(u1)
def count_hash(text):
    text = "".join(text)
    u = re.findall('#[a-zA-Z0-9]+',text)
    return len(u)



def clear_text(text):
    def decontracted(phrase):
       # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase

    text = decontracted(text)
    
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
    text = re.sub('[^a-zA-Z]+',' ',text)
    

    
    
    return text

    
train["urls"] = train["tweet"].apply(lambda x: get_url(x))
train["n_hash"] = train["tweet"].apply(lambda x: count_hash(x))
train['tweet'] = train['tweet'].apply(lambda x: clear_text(x))


train["word_count"] = train["tweet"].apply(lambda x: len([t for t in x.split() if len(t)>1]))
train["stop_count"] = train["tweet"].apply(lambda x: len([t for t in x.split() if t in stopwords_]))
train["tweet"] = train["tweet"].apply(lambda x: x.lower())

In [64]:
a = "i'm ashihs 't' won\'t"
re.sub("won't","will not",a)

"i'm ashihs 't' will not"

In [72]:
train.head()

,id,label,tweet,urls,n_hash,word_count,stop_count
0,1,0,fingerprint pregnancy test android apps beautiful cute health igers iphoneonly iphonesia iphone,1,11,12,0
1,2,0,finally a transparant silicon case thanks to my uncle yay sony xperia s sonyexperias,1,5,12,3
2,3,0,we love this would you go talk makememories unplug relax iphone smartphone wifi connect,1,8,14,2
3,4,0,i am wired i know i am george i was made that way iphone cute daventry home,1,4,13,4
4,5,1,what amazing service apple will not even talk to me about a question i have unless i pay them for their stupid support,0,0,20,10


In [75]:
train.groupby(["label"])["n_hash","word_count","stop_count"].describe()

n_hash                                               word_count  \
        count      mean       std  min  25%  50%  75%   max      count   
label                                                                    
0      5894.0  7.319817  4.850001  0.0  4.0  7.0  9.0  33.0  5894.0      
1      2026.0  1.788746  2.095920  0.0  0.0  1.0  2.0  26.0  2026.0      

                                                        stop_count            \
            mean       std  min   25%   50%   75%   max      count      mean   
label                                                                          
0      15.293180  6.097452  1.0  11.0  13.0  17.0  48.0  5894.0     2.559722   
1      16.876604  4.829598  7.0  13.0  16.0  20.0  52.0  2026.0     6.580948   

                                           
            std  min  25%  50%  75%   max  
label                                      
0      2.714870  0.0  1.0  2.0  4.0  25.0  
1      3.307845  0.0  4.0  6.0  9.0  27.0

In [73]:
train['tweet'] = train['tweet'].apply(lambda x: nlp(x))

,id,label,tweet,urls,n_hash,word_count,stop_count
0,1,0,fingerprint pregnancy test android apps beautiful cute health igers iphoneonly iphonesia iphone,1,11,12,0
1,2,0,finally a transparant silicon case thanks to my uncle yay sony xperia s sonyexperias,1,5,12,3
2,3,0,we love this would you go talk makememories unplug relax iphone smartphone wifi connect,1,8,14,2
3,4,0,i am wired i know i am george i was made that way iphone cute daventry home,1,4,13,4
4,5,1,what amazing service apple will not even talk to me about a question i have unless i pay them for their stupid support,0,0,20,10
5,6,1,iphone software update fucked up my phone big time stupid iphones,0,0,11,2
6,7,0,happy for us instapic instadaily us sony xperia xperiaz,1,6,9,1
7,8,0,new type c charger cable uk bay amazon etsy new year rob cross toby young evemun mcmafia taylor spectre newyear starting recipes technology samsunggalaxys iphonex pic twitter com pjiwq wtc,1,16,29,0
8,9,0,bout to go shopping again listening to music iphone justme music likeforlike followforfollow,1,5,13,3
9,10,0,photo fun selfie pool water sony camera picoftheday sun instagood boy cute outdoor,1,12,13,0
